*Analytical Information Systems*

# Worksheet 5 - Big Data and Streaming

Matthias Griebel<br>
Lehrstuhl für Wirtschaftsinformatik und Informationsmanagement

SS 2020

## MapReduce

[MapReduce](https://en.wikipedia.org/wiki/MapReduce) is a programming model and an associated implementation for processing and generating big data sets with a parallel, distributed algorithm on a cluster.

Let's have a look at the word count example from the lecture again

<img src="http://wi-wiki.de/lib/exe/fetch.php?cache=&w=899&h=417&tok=68959d&media=bigdata:mapreducewordcountoverview1.png" style="width:50%">

1. __Input__

1. __Splitting__: Prepare the Map() input

1. __Mapping__: Run the user-provided Map() code. Each worker node applies the map function to the local data, and writes the output to a temporary storage.

1. __Shuffling__: "Shuffle" the Map output to the Reduce processors. 

1. __Reduce__: Run the user-provided Reduce() code. The Reduce processors process each group of output data, per key, in parallel.

1. __Final result__: Produce the final output – the MapReduce system collects and sorts all the Reduce output

### MapReduce Libraries

MapReduce libraries have been written in many programming languages, with different levels of optimization. 
- A popular open-source implementation that has support for distributed shuffles is part of Apache Hadoop.
- [RHadoop](https://github.com/RevolutionAnalytics/RHadoop/wiki) is a collection of five R packages that allow users to manage and analyze data with Apache Hadoop. 
    - using RHadoop requires a Java and Hadoop installation, the Hadoop Distributed File System, etc.

Thus, we will only examplify the MapReduce algorithm using basic R and the `tidyverse`:

### Examplary R MapReduce Word Count Implementation

__Defining the map function__

The map function breaks the line into words and outputs a key/value pair for each word.

In [0]:
library(tidyverse)

In [0]:
count_words <- function(line){
    line %>%
            str_split(" ",simplify=FALSE) %>%
            unlist() %>%
            tibble(key=., value=1)
}

__Defining the reduce function__

In the word count example, the Reduce function sums the word counts and generates a single output of the word and the final sum.

In [0]:
reduce_count <- function(df){
    df %>%
        summarise(key=key[1],
                  count=sum(value))
}

__Going through the MapReduce steps__

__1. Input__

In [0]:
Input =  "Deer Bear River\nCar Car River\nDeer Car Bear"
Input

[1] "Deer Bear River\nCar Car River\nDeer Car Bear"

__2. Splitting__

We will split the input by line ('\n' indicates a new line)

In [0]:
Input %>%
    str_split("\n",simplify=FALSE) %>% unlist

[1] "Deer Bear River" "Car Car River"   "Deer Car Bear"

2. Mapping

In [0]:
Input %>%
    str_split("\n",simplify=FALSE) %>% unlist %>%
    map(count_words)

key,value
<chr>,<dbl>
Deer,1
Bear,1
River,1
key,value
<chr>,<dbl>
Car,1
Car,1
River,1
key,value


4. Shuffling

In [0]:
Input %>%
    str_split("\n",simplify=FALSE) %>% unlist %>%
    map(count_words) %>% 
    map_df(rbind) %>% group_split(key)

key,value
<chr>,<dbl>
Bear,1
Bear,1
key,value
<chr>,<dbl>
Car,1
Car,1
Car,1
key,value
<chr>,<dbl>


5. Reducing

In [0]:
Input %>%
    str_split("\n",simplify=FALSE) %>% unlist %>%
    map(count_words) %>% 
    map_df(rbind) %>% group_split(key) %>%
    map(reduce_count)

key,count
<chr>,<dbl>
Bear,2
key,count
<chr>,<dbl>
Car,3
key,count
<chr>,<dbl>
Deer,2
key,count
<chr>,<dbl>


5. Merge and sort

In [0]:
Input %>%
    str_split("\n",simplify=FALSE) %>% unlist %>%
    map(count_words) %>% 
    map_df(rbind) %>% group_split(key) %>%
    map(reduce_count) %>%
    map_df(cbind) %>% arrange(desc(count))

key,count
<chr>,<dbl>
Car,3
Bear,2
Deer,2
River,2


__Doing it the undistributed tidyverse way__

In [0]:
Input %>%
    str_replace_all("\n", " ") %>%
    str_split(" ",simplify=FALSE) %>% unlist %>% 
    tibble(key=.) %>%
    group_by(key) %>%
    summarize(count=n()) %>%  arrange(desc(count))

key,count
<chr>,<int>
Car,3
Bear,2
Deer,2
River,2


## Stream Processing

__Credits__

- Jure Leskovec, Stanford University, http://web.stanford.edu/class/cs246/slides/15-streams1.pdf
- Michael Freedman, Princeton University, https://www.cs.princeton.edu/courses/archive/fall16/cos418/docs/L22-stream-processing.pdf

### Data Streams

- In many data mining situations, we do not know the entire data set in advance
- We can think of the data as infinite and non-stationary (the distribution changes over time)
- Stream Management is important when the input rate is controlled externally:
    - Google queries
    - Twitter or Facebook status updates

__The Stream Model__

- Input elements enter at a rapid rate, at one or more input ports (i.e., streams)
    - We call elements of the stream tuples
- The system cannot store the entire stream accessibly

    
How do you make critical calculations about the stream using a limited amount of (secondary) memory?

### Basic Stream Operators

__Stateless conversion__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/sc_ctoF.png" width="30%">

- Convert Celsius temperature to Fahrenheit: __emit__ (input * 9 / 5) + 3

__Stateless filtering__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/sc_sf.png" width="30%">

Function can filter inputs: –if(input>threshold) {__emit__ input}

__Stateful conversion__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/sc_ewa.png" width="30%">

Compute EWMA of Fahrenheit temperature:
- new_temp = ⍺ * ( CtoF(input) ) + (1- ⍺) * last_temp
- last_temp = new_temp – emit new_temp
- emit new_temp

__Aggregation (stateful)__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/sc_agg.png" width="30%">

E.g.,Average value per window
- Window can be # elements (10) or time (1s)
- Windows can be disjoint (every 5s)
- Windows can be “tumbling” (5s window every 1s)

__Stream processing as chain__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/sc_chain.png" width="30%">

__Stream processing as directed graph__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/sc_chain.png" width="30%">

### The challenge of stream processing for BIG DATA

Large amounts of data to process in realtime

__Examples__:
- Social network trends (#trending)
- Intrusion detection systems (networks, datacenters)
- Sensors: Detect earthquakes by correlating vibrations of millions of smartphones
- Fraud detection
    - Visa: 2000 txn / sec on average, peak ~47,000 / sec

__Stateless operations: trivially parallelized__

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/scale_out.png" width="30%">

__State complicates parallelization__


- Need to join results across parallel computations

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/agg_par.png" width="30%">


__Parallelization complicates fault-tolerance__


<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/fault.png" width="30%">


__We can parallelize joins__

- using partitioned hash joins
- but agian, complicates fault-tolerance

<img src="http://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/figures/05/par_joins.png" width="30%">



### Stream Processing frameworks

Different frameworks handle these challenges differently

- Record acknowledgement (Storm)
- Micro-batches (Spark Streaming, Storm Trident) 
- Transactional updates (GoogleClouddataflow) 
- Distributed snapshots (Flink)

### Streaming data with R

__The `sparklyr` interface for Spark Streaming__

from the official [Website](https://spark.rstudio.com/guides/streaming/):

Spark Streaming makes it easy to build scalable fault-tolerant streaming applications. Because is part of the Spark API, it is possible to re-use query code that queries the current state of the stream, as well as joining the streaming data with historical data. Please see Spark’s official documentation for a deeper look into Spark Streaming.

The sparklyr interface provides the following:

- Ability to run dplyr, SQL, spark_apply(), and PipelineModels against a stream
- Read in multiple formats: CSV, text, JSON, parquet, Kafka, JDBC, and orc
- Write stream results to Spark memory and the following file formats: CSV, text, JSON, parquet, Kafka, JDBC, and orc
- An out-of-the box graph visualization to monitor the stream
- A new reactiveSpark() function, that allows Shiny apps to poll the contents of the stream create Shiny apps that are able to read the contents of the stream


#### Interacting with a stream

A good way of looking at the way how Spark streams update is as a three stage operation:

1. __Input__ - Spark reads the data inside a given folder. The folder is expected to contain multiple data files, with new files being created containing the most current stream data.

1. __Processing__ - Spark applies the desired operations on top of the data. These operations could be data manipulations (dplyr, SQL), data transformations (sdf operations, PipelineModel predictions), or native R manipulations (spark_apply()).

1. __Output__ - The results of processing the input files are saved in a different folder.

#### `sparklyr` Example

__Install requirements__

This can take a few minutes...

In [0]:
system("apt-get install openjdk-8-jdk-headless -qq > /dev/null")
Sys.setenv(JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64")
install.packages(c("sparklyr", "future"))
library(sparklyr)
spark_install()
library(future)
library(tidyverse)

1. Open the Spark connection

In [0]:
sc <- spark_connect(master = "local")

Optional step. This resets the input and output folders. It makes it easier to run the code multiple times in a clean manner.

In [0]:
if(file.exists("source")) unlink("source", TRUE)
if(file.exists("source-out")) unlink("source-out", TRUE)

2. Produce a single test file inside the “source” folder. This allows the “read” function to infer CSV file definition.

In [4]:
stream_generate_test(iterations = 1)
list.files("source")

[1] "stream_1.csv"

3. Point the stream reader to the folder where the streaming files will be placed. 

In [0]:
read_folder <- stream_read_csv(sc, "source") 

4. Process stream function: The processing starts with the read_folder variable that contains the input stream. It coerces the integer field x, into a type double. This is because the next function, ft_binarizer() does not accept integers. The binarizer determines if x is over 400 or not. This is a good illustration of how dplyr can help simplify the manipulation needed during the processing stage.

In [0]:
process_stream <- read_folder %>%
  mutate(x = as.double(x)) %>%
  ft_binarizer(
    input_col = "x",
    output_col = "over",
    threshold = 400
  )

4. The output writer is what starts the streaming job. It will start monitoring the input folder, and then write the new results in the “source-out” folder. 

In [0]:
write_output <- stream_write_csv(process_stream, "source-out")

5. The test generation function will run 100 files every 0.2 seconds. To run the tests “out-of-sync” with the current R session, the future package is used.

In [0]:
invisible(future(stream_generate_test(interval = 0.2, iterations = 100)))

6. The “source-out” folder can be treated as a if it was a single table within Spark. Using spark_read_csv(), the data can be mapped, but not brought into memory (memory = FALSE). This allows the current results to be further analyzed using regular dplyr commands.

In [13]:
spark_read_csv(sc, "stream", "source-out", memory = FALSE)

# Source: spark<?> [?? x 2]
   over     n
  <dbl> <dbl>
1     0    12

## Exercises

### 1. MapReduce

__Sales analysis__

You need to run a company-wide sales analysis. Your company uses a MapReduce system to handle the massive transaction data.

We will have a look at the data first:


In [0]:
sales <- read_csv('https://raw.githubusercontent.com/wi3jmu/AIS2020/master/notebooks/data/sales.csv')
sales %>% head(10)

Parsed with column specification:
cols(
  date = col_date(format = ""),
  customerID = col_double(),
  productID = col_double(),
  payment = col_character(),
  amount = col_double(),
  price = col_double(),
  cost = col_double(),
  category = col_character()
)



date,customerID,productID,payment,amount,price,cost,category
<date>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
2017-01-16,64292,8403,paypal,3,560.74,234.89,emergency
2017-08-16,41174,7234,paypal,3,351.14,171.11,specialty
2017-10-26,49737,32738,paypal,3,343.38,105.14,emergency
2017-11-24,24021,70159,cash,2,905.96,345.40,emergency
2017-02-13,78762,2002,cash,2,799.99,407.30,emergency
2017-07-18,79148,86205,credit card,1,284.07,132.35,emergency
2017-08-23,79148,40784,cash,3,125.79,47.53,specialty
2017-11-06,23090,16224,paypal,3,85.77,36.61,specialty
2017-09-28,12307,82560,credit card,2,658.88,330.44,emergency


Define the corresponding Map and Reduce functions:

__Map__: Calculates the total profit for each product id within each subset

In [0]:
calculate_profit <- function(df){
    df %>%
        # Write your code here 
}

__Reduce__: Adds up the profit for each different product id

In [0]:
reduce_profit <- function(df){
    df %>%
        # Write your code here 
}

In [0]:
sales %>% #Input
    split(sample(rep(1:5, 1000))) %>% #Splitting
    map(calculate_profit) %>% #Mapping
    map_df(rbind) %>% group_split(productID)%>% #Shuffling
    map(reduce_profit) %>% #Reduce
    map_df(cbind) %>% arrange(desc(total_profit)) %>%  #Merge and Sort
    head(10) #Display only top 10

,productID,total_profit
,<dbl>,<dbl>
1,43446,171232.25
2,24221,111657.54
3,60974,88118.82
4,70159,84644.56
5,89266,73408.04
6,86064,68414.58
7,9947,64715.40
8,61070,64571.40
9,62077,63238.50


### 2. Stream Processing

1. Why do stateful operation complicate parallelization?

In [0]:
 # Write your answer here

2. Why do parallelization operations complicate fault-tolerance?

In [0]:
 # Write your answer here